In [ ]:
#Training the model on 30k datasets.

import random
import numpy as np
from Bio import SeqIO
import gzip
import os
import pickle
from tensorflow import set_random_seed
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, Dropout
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd
import matplotlib.pyplot as plt

#Set random seeds for reproducibility.
np.random.seed(4)
random.seed(5)
set_random_seed(4) 



def load_data(path):   
    data = gzip.open(os.path.join(path,"sequences.fa.gz"),"rt")
    return data


def get_seq(protein, t_data, training_set_number): 
    if t_data == "train":     
        training_data = load_data("datasets/clip/%s/30000/training_sample_%s"% (protein, training_set_number))
        x_train = np.zeros((30000,101,4))          
    
    elif t_data == "test":          
        training_data = load_data("datasets/clip/%s/30000/test_sample_%s"% (protein, training_set_number))
        x_train = np.zeros((10000,101,4))      
    r = 0 
    for record in SeqIO.parse(training_data,"fasta"):
        sequence = list(record.seq)                
        nucleotide = {'A' : 0, 'T' : 1, 'G' : 2, 'C' : 3, 'N' : 4} 
        num_seq = list() #Encoded sequence
        for i in range(0,len(sequence)):
                num_seq.append(nucleotide[sequence[i]])

        X = np.zeros((1,len(num_seq),4))
        for i in range (len(num_seq)):
                if num_seq[i] <= 3:
                    X[:,i,num_seq[i]] = 1               
        x_train[r,:,:] = X
        r = r + 1
    
    return x_train


def get_class(protein, t_data,training_set_number):
    y_train = []
    if t_data == 'train':
        data = load_data("datasets/clip/%s/30000/training_sample_%s"% (protein, training_set_number))
    elif t_data == 'test':
        data = load_data("datasets/clip/%s/30000/test_sample_%s"% (protein, training_set_number))
    for record in SeqIO.parse(data,"fasta"):
        v = int((record.description).split(":")[1])
        # [1,0] if there was no observed binding and [0,1] for sequences where binding was observed.
        y_train.append([int(v == 0), int(v != 0)])

    y_train = np.array(y_train)
    return y_train


def get_cobinding(protein, t_data, training_set_number):
    if t_data == "train":
        with gzip.open(("datasets/clip/%s/30000/training_sample_%s/matrix_Cobinding.tab.gz"% (protein, training_set_number)), "rt") as f:
            cobinding_data = np.loadtxt(f, skiprows=1)       
        cobinding = np.zeros((30000,101,int(cobinding_data.shape[1]/101)),dtype=np.int)    
    elif t_data == "test":
        with gzip.open(("datasets/clip/%s/30000/test_sample_%s/matrix_Cobinding.tab.gz"% (protein, training_set_number)), "rt") as f:
            cobinding_data = np.loadtxt(f, skiprows=1) 
        cobinding = np.zeros((10000,101,int(cobinding_data.shape[1]/101)),dtype=np.int)
    for n in range(0,cobinding_data.shape[1],101):
        a = cobinding_data[:,n:(n+101)]
        cobinding[:,:,int(n/101)] = a
    
    return cobinding
    

def get_region (protein, t_data, training_set_number):
    if t_data == "train":
        with gzip.open(("datasets/clip/%s/30000/training_sample_%s/matrix_RegionType.tab.gz"% (protein, training_set_number)), "rt") as f:
            region_data = np.loadtxt(f, skiprows=1)
        region = np.zeros((30000,101,int(region_data.shape[1]/101)),dtype=np.int)
                             
    elif t_data == "test":
        with gzip.open(("datasets/clip/%s/30000/test_sample_%s/matrix_RegionType.tab.gz"% (protein, training_set_number)), "rt") as f:
            region_data = np.loadtxt(f, skiprows=1) 
        region = np.zeros((10000,101,int(region_data.shape[1]/101)),dtype=np.int)
    for n in range(0,region_data.shape[1],101):
        a = region_data[:,n:(n+101)]
        region[:,:,int(n/101)] = a
        
    return region


def get_fold (protein, t_data, training_set_number):
    if t_data == "train":
        with gzip.open(("datasets/clip/%s/30000/training_sample_%s/matrix_RNAfold.tab.gz"% (protein, training_set_number)), "rt") as f:
            fold_data = np.loadtxt(f, skiprows=1) 
        fold = np.zeros((30000,101,int(fold_data.shape[1]/101)), dtype=np.int)
                             
    elif t_data == "test":
        with gzip.open(("datasets/clip/%s/30000/test_sample_%s/matrix_RNAfold.tab.gz"% (protein, training_set_number)), "rt") as f:
            fold_data = np.loadtxt(f, skiprows=1) 
        fold = np.zeros((10000,101,int(fold_data.shape[1]/101)),dtype=np.int)




    for n in range(0,fold_data.shape[1],101):
        a = fold_data[:,n:(n+101)]
        fold[:,:,int(n/101)] = a
    
    
    return fold

def load_data_sources(protein, t_data, training_set_number, *args):
    X = np.array([])
    data_sources = []
    for arg in args:
        
        if arg == 'KMER':
            if X.size == 0:
                X = get_seq(protein, t_data, training_set_number)
            else: 
                X = np.dstack((X, get_seq(protein, t_data, training_set_number)))
        if arg == 'RNA': 
            if X.size == 0:
                X = get_fold(protein, t_data, training_set_number)
            else: 
                X = np.dstack((X, get_fold(protein, t_data, training_set_number)))
        if arg == 'RG':   
            if X.size == 0:
                X = get_region(protein, t_data, training_set_number)
            else: 
                X = np.dstack((X, get_region(protein, t_data, training_set_number)))
        if arg == 'CLIP': 
            if X.size == 0:
                X = get_cobinding(protein, t_data, training_set_number)
            else: 
                X = np.dstack((X, get_cobinding(protein, t_data, training_set_number)))
        data_sources.append(arg)
        
    data_sources = ','.join(data_sources)
    return data_sources, X


In [16]:
protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19", "3_HITSCLIP_Ago2_binding_clusters", "4_HITSCLIP_Ago2_binding_clusters_2", "5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1", "7_CLIP-seq-eIF4AIII_2", "8_PARCLIP_ELAVL1_hg19", "9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome",
                "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19", "26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]
scores_dict = {}
for protein in protein_list:
#     print(protein)
    for training_set_number in range(3):
        with open("results/set_%s/%s/predictions" % (training_set_number, protein), "rb") as predictions_file:
            loaded_predictions = np.load(predictions_file)
        with open("../iONMF/results/set_%s/%s_predictions" % (training_set_number, protein), "rb") as ionmf_file:
            ionmf_loaded = np.load(ionmf_file)
        ideep_loaded = np.loadtxt(
            f"../iDeep/predictions_{training_set_number}/{protein}", delimiter="\n", unpack=False)
        y_test = get_class(protein, "test", training_set_number)

        if training_set_number == 0:
            imcnn_predictions = loaded_predictions[:, 0:1]
            imcnn_test = y_test[:, 0:1]
            ideep_predictions = ideep_loaded
            ideep_test = y_test[:, 1]
            ionmf_predictions = ionmf_loaded

        else:
            imcnn_predictions = np.concatenate(
                (imcnn_predictions, loaded_predictions[:, 0:1]))
            imcnn_test = np.concatenate((imcnn_test, y_test[:, 0:1]))
            ideep_predictions = np.concatenate(
                (ideep_predictions, ideep_loaded))
            ideep_test = np.concatenate((ideep_test, y_test[:, 1]))
            ionmf_predictions = np.concatenate(
                (ionmf_predictions, ionmf_loaded))

    imcnn_fpr, imcnn_tpr, _ = roc_curve(imcnn_test, imcnn_predictions)
    ideep_fpr, ideep_tpr, _ = roc_curve(ideep_test, ideep_predictions)
    ionmf_fpr, ionmf_tpr, _ = roc_curve(ideep_test, ionmf_predictions)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.plot(imcnn_fpr, imcnn_tpr, label='imcnnRNA')
    ax.plot(ideep_fpr, ideep_tpr, label='iDeep')
    ax.plot(ionmf_fpr, ionmf_tpr, label='iONMFF')

    ax.plot([0, 1], [0, 1], 'k--')
    ax.legend(loc='lower right')
    ax.set_xlabel('False positive rate')
    ax.set_ylabel('True positive rate')
    ax.set_title('ROC curves of %s' % protein)
    plt.savefig(f"figures/ROC/{protein}.png")
#     plt.show()
    plt.close()

In [ ]:
from sklearn.calibration import calibration_curve

    
protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]
for protein in protein_list:
    average_score = np.zeros(3)
#     print(protein)
    complete_ideep_prediction = []
    y_scores = []
    total_y_test = []
    
    for training_set_number in range (3):
#         print("Training set number " + str(training_set_number))
        #Load model predictions.
        with open ("results/set_%s/%s/predictions" % (training_set_number, protein), "rb") as predictions_file,\
            open ("ideep_results/predictions%s/%s" % (training_set_number, protein), "rb") as ideep_file:
            loaded_predictions = np.load(predictions_file)
            ideep_prediction = np.loadtxt(ideep_file, delimiter="\n", unpack=False)
            ideep_prediction = np.array(ideep_prediction)
            complete_ideep_prediction = np.append(complete_ideep_prediction, [abs(i-1) for i in ideep_prediction])
            # Evaluate model performance.
            y_scores = np.append(y_scores, loaded_predictions [:,0:1])
            y_test = get_class(protein,"test",training_set_number)
            total_y_test = np.append(total_y_test, y_test [:,0:1])
#     print(len(y_scores))
    y_scores.flatten()
    total_y_test.flatten()
    complete_ideep_prediction.flatten()
    fraction_of_positives, mean_predicted_value = calibration_curve(total_y_test, y_scores, n_bins=10)
    ideep_frp, ideep_mpv = calibration_curve(total_y_test, complete_ideep_prediction, n_bins=10)
    
    fig = plt.figure(figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))
    
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    ax1.plot(mean_predicted_value, fraction_of_positives, label="imcnnRNA")
    ax1.plot(ideep_mpv, ideep_frp, label="iDeep")
    ax1.set_ylabel("Fraction of positives")
    ax1.set_xlabel("Mean predicted value")            
    ax1.legend()

    ax2.hist(y_scores, range=(0, 1), bins=10, histtype="step", lw=2, label="imcnnRNA")
    ax2.hist(complete_ideep_prediction, range=(0, 1), bins=10, histtype="step", lw=2, label="iDeep")
    ax2.hist(total_y_test, range=(0, 1), bins=10, histtype="step", lw=2, color="black", alpha=0.7, ls="--", label="Dataset")
    ax2.legend(loc="upper left")
    ax2.set_ylabel("Count")
    ax2.set_xlabel("Mean predicted value")
    
    plt.tight_layout()
    
    plt.savefig("results/calibration_curves/{}_calibration_curve.png".format(protein))
#     plt.show()
    plt.close()

In [15]:
from IPython.display import Latex

protein_names = ['1 Ago/EIF', '2 Ago2-MNase', '3 Ago2-1', '4 Ago2-2', '5 Ago2', '6 eIF4AIII-1', '7 eIF4AIII-2', '8 ELAVL1-1', '9 ELAVL1-MNase', '10 ELAVL1A', '11 ELAVL1-2', '12 ESWR1', '13 FUS', '14 Mut FUS', '15 IGFBP1-3', '16 hnRNPC-1', '17 hnRNPC-2', '18 hnRNPL-1', '19 hnRNPL-2', '20 hnRNPL-like', '21 MOV10', '22 Nsun2', '23 PUM2', '24 QKI', '25 SRSF1' , '26 TAF15', '27 TDP-43', '28 TIA1', '29 TIAL1', '30 U2AF2', '31 U2AF2(KD)']


def makeplot(figname, figlabel, figcaption):
    strLatex=r"""
    \begin{figure}
        \begin{center}
            \includegraphics[max size={0.9\linewidth}{0.4\paperheight}]{%s}
            \caption{%s}
            \label{%s}
        \end{center}
    \end{figure}"""%(figname, figcaption, figlabel) 
    return display(Latex(strLatex)) 

for protein, name in zip(protein_list, protein_names):
    figname = f"figures/ROC/{protein}.png"
    figlabel = f'fig:ROC{name}'
    figcaption = f'ROC curves for {name}'
    makeplot(figname, figlabel, figcaption)
    
for protein, name in zip(protein_list, protein_names):
    figname = f"figures/calibration_curves/{protein}_calibration_curve.png"
    figlabel = f'fig:calibration{name}'
    figcaption = f'Calibration curves for {name}'
    makeplot(figname, figlabel, figcaption)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>